**Requirements**

pip install krakenex  
pip install pykrakenapi

In [1]:
from krakenex import API
from pykrakenapi import KrakenAPI

k = KrakenAPI(API())

In [2]:
markets = ['XRP','BTC']

In [3]:
from datetime import datetime
from os import path
import pandas as pd

getSince = { 'default': datetime(2020,1,1).timestamp() }

for market in markets:
    marketFile = market + '.csv'
    if path.exists(marketFile):
        df = pd.read_csv(marketFile)
        getSince[market] = df.iloc[-1].time
    else:
        getSince[market] = getSince['default']
        
print(getSince)

{'default': 1577847600.0, 'XRP': 1649871360, 'BTC': 1649871420}


In [5]:
# Get open high low close data since date
from time import mktime, sleep
from datetime import datetime
from os import path

# interval = {1, 5, 15, 30, 60, 240, 1440, 10080, 21600} [min]

def extendCSV(df,fileName):
    df.to_csv(fileName, mode='a', header=not path.exists(fileName))

while True:    
    for im, market in enumerate(markets):
        print('Retreiving '+market+' at',datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
        print('\t last record: \t',datetime.utcfromtimestamp(getSince[market]))
    
        ohlc, getSince[market] = k.get_ohlc_data(pair = market+'USD',interval=1,since = getSince[market],ascending = True)
        
        print('\t first retreived: \t',datetime.utcfromtimestamp(ohlc.iloc[0].time))
        print('\t last retreived: \t',datetime.utcfromtimestamp(getSince[market]))
    
        extendCSV(ohlc.iloc[:-1], market+'.csv')
        sleep(1)
    
    print(' ... waiting 0', end = "")
    for i in range(6):
        sleep(600)
        print(',', 10*(i+1), end = "")
    print(', 60, done! let\'s go again')

Retreiving XRP at 13/04/2022 19:55:29
	 last record: 	 2022-04-13 23:52:00
	 first retreived: 	 2022-04-13 23:53:00
	 last retreived: 	 2022-04-13 23:54:00
Retreiving BTC at 13/04/2022 19:55:31
	 last record: 	 2022-04-13 23:52:00
	 first retreived: 	 2022-04-13 23:53:00
	 last retreived: 	 2022-04-13 23:54:00
 ... waiting 0

KeyboardInterrupt: 